In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import torchmetrics
import pytorch_lightning as pl

In [2]:
# hyperparams
in_channels = 3
num_classes = 10
learning_rate = 0.001
batch_size = 512
num_epochs = 10

In [3]:
image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

In [4]:
# train_dataset = datasets.ImageFolder("/Users/pepe/dev/upb/topics/datasets/cats/train", transform=image_transforms)
train_dataset = datasets.ImageFolder("/home/pepe/dev/upb/topics/cats/train", transform=image_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=24, persistent_workers=True)
# test_dataset = datasets.ImageFolder("/Users/pepe/dev/upb/topics/datasets/cats/test", transform=image_transforms)
test_dataset = datasets.ImageFolder("/home/pepe/dev/upb/topics/cats/test", transform=image_transforms)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=24, persistent_workers=True)


In [5]:
class CNN(pl.LightningModule):
    def __init__(self, in_channels=3, num_classes=5):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.fc1 = nn.Linear(64 * 14 * 14, num_classes)

        self.loss_fn = nn.CrossEntropyLoss()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            self.pool,
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            self.pool,
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            self.pool,
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            self.pool,
        )
        self.head = nn.Sequential(
            self.fc1,
        )

        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = self.features(x)
        # flatten
        x = x.reshape(x.shape[0], -1)
        x = self.head(x)
        return x

    def training_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)

        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy, "train_f1_score": f1_score},
            on_step=False,
            on_epoch=True,
            prog_bar=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)

        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy, "test_f1_score": f1_score},
            on_step=False,
            on_epoch=True,
            prog_bar=True
        )
        return loss

In [6]:
class MyResnet18(pl.LightningModule):
    def __init__(self, num_classes=5, freeze_backbone=False):
        super().__init__()
        self.weights = torchvision.models.ResNet18_Weights.DEFAULT
        self.backbone = torchvision.models.resnet18(weights=self.weights)
        self.preprocess = self.weights.transforms()
        if freeze_backbone:
            for param in self.parameters():
                param.requires_grad = False
        self.fc = nn.Linear(512, num_classes)
        self.backbone.fc = self.fc
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)

        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy, "train_f1_score": f1_score},
            on_step=False,
            on_epoch=True,
            prog_bar=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)

        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy, "test_f1_score": f1_score},
            on_step=False,
            on_epoch=True,
            prog_bar=True
        )
        return loss

    def _common_step(self, batch, batch_idx):
        x, y = batch
        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        return loss, scores, y

    def predict_step(self, batch, batch_idx):
        x, y = batch
        scores = self.forward(x)
        preds = torch.argmax(scores, dim=1)
        return preds

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

In [7]:
model = MyResnet18(num_classes=num_classes)
# formato:      batch_size, n_channels, x, y
x = torch.randn(64, 3, 224, 224)
print(x.shape)
print(model(x).shape)

torch.Size([64, 3, 224, 224])
torch.Size([64, 10])


In [8]:
torch.set_float32_matmul_precision('high')
trainer = pl.Trainer(
    accelerator="gpu", # para GPUs Nvidia: "gpu"
    devices=1,          
    min_epochs=1,
    max_epochs=num_epochs,
    precision="bf16-mixed"
)
trainer.fit(model, train_loader, test_loader)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/pepe/miniconda3/envs/topics/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                | Params
---------------------------------------------------
0 | backbone   | ResNet              | 11.2 M
1 | preprocess | ImageClassification | 0     
2 | fc         | Line

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/pepe/miniconda3/envs/topics/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [9]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.9399999976158142
      test_f1_score         0.9399999976158142
        test_loss           0.18903234601020813
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.18903234601020813,
  'test_accuracy': 0.9399999976158142,
  'test_f1_score': 0.9399999976158142}]

In [17]:
from torchvision.io import read_image
model.eval()
img = read_image("tiger1.jpg")
batch = model.preprocess(img).unsqueeze(0)
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = train_dataset.classes[class_id]
print(prediction, class_id, score, category_name)

tensor([0.0049, 0.0019, 0.0572, 0.0037, 0.0697, 0.0190, 0.0073, 0.0029, 0.0085,
        0.8249], grad_fn=<SoftmaxBackward0>) 9 0.8249470591545105 TIGER
